In [1]:
import pandas as pd
import numpy as np
import requests
from io import StringIO
import re
from datetime import datetime
import yfinance as yf
import time

In [2]:

def get_ipos() -> pd.DataFrame:
    """
    Fetch IPO data for the given year from stockanalysis.com.
    """
    url = f"https://stockanalysis.com/ipos/withdrawn/"
    headers = {
        'User-Agent': (
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
            'AppleWebKit/537.36 (KHTML, like Gecko) '
            'Chrome/58.0.3029.110 Safari/537.3'
        )
    }

    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()

        # Wrap HTML text in StringIO to avoid deprecation warning
        # "Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object."
        html_io = StringIO(response.text)
        tables = pd.read_html(html_io)

        if not tables:
            raise ValueError(f"No tables found.")

        return tables[0]

    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
    except ValueError as ve:
        print(f"Data error: {ve}")
    except Exception as ex:
        print(f"Unexpected error: {ex}")

    return pd.DataFrame()

In [3]:
ipos_withdrawals = get_ipos()
ipos_withdrawals.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Symbol          100 non-null    object
 1   Company Name    100 non-null    object
 2   Price Range     100 non-null    object
 3   Shares Offered  100 non-null    object
dtypes: object(4)
memory usage: 3.3+ KB


In [4]:
# function to categorize companies by class

def classify_company(name):
    name = str(name).lower()
    words = re.findall(r'\b\w+\b', name)  # wyłapuje tylko czyste słowa, ignorując przecinki/kropki

    if 'acquisition' in words and ('corp' in words or 'corporation' in words):
        return 'Acq.Corp'
    elif 'inc' in words or 'incorporated' in words:
        return 'Inc'
    elif 'group' in words:
        return 'Group'
    elif 'ltd' in words or 'limited' in words:
        return 'Limited'
    elif 'holdings' in words:
        return 'Holdings'
    else:
        return 'Other'

In [5]:
# Create a new column
ipos_withdrawals['Company Class'] = ipos_withdrawals['Company Name'].apply(classify_company)

In [6]:
ipos_withdrawals

,Symbol,Company Name,Price Range,Shares Offered,Company Class
0,ODTX,"Odyssey Therapeutics, Inc.",-,-,Inc
1,UNFL,"Unifoil Holdings, Inc.",$3.00 - $4.00,2000000,Inc
2,AURN,"Aurion Biotech, Inc.",-,-,Inc
3,ROTR,"PHI Group, Inc.",-,-,Inc
4,ONE,One Power Company,-,-,Other
...,...,...,...,...,...
95,FHP,"Freehold Properties, Inc.",-,-,Inc
96,CHO,Chobani Inc.,-,-,Inc
97,IFIT,iFIT Health & Fitness Inc.,$18.00 - $21.00,30769231,Inc
98,GLGX,"Gerson Lehrman Group, Inc.",-,-,Inc


In [7]:
def parse_avg_price(price_range):
    if not isinstance(price_range, str) or '-' in price_range.strip() and price_range.strip() == '-':
        return None

    # Pull out numbers 
    numbers = re.findall(r'\d+(?:\.\d+)?', price_range)

    if not numbers:
        return None
    numbers = list(map(float, numbers))
    return sum(numbers) / len(numbers)

In [8]:
ipos_withdrawals['Avg. price'] = ipos_withdrawals['Price Range'].apply(parse_avg_price)

In [9]:
ipos_withdrawals

,Symbol,Company Name,Price Range,Shares Offered,Company Class,Avg. price
0,ODTX,"Odyssey Therapeutics, Inc.",-,-,Inc,NaN
1,UNFL,"Unifoil Holdings, Inc.",$3.00 - $4.00,2000000,Inc,3.5
2,AURN,"Aurion Biotech, Inc.",-,-,Inc,NaN
3,ROTR,"PHI Group, Inc.",-,-,Inc,NaN
4,ONE,One Power Company,-,-,Other,NaN
...,...,...,...,...,...,...
95,FHP,"Freehold Properties, Inc.",-,-,Inc,NaN
96,CHO,Chobani Inc.,-,-,Inc,NaN
97,IFIT,iFIT Health & Fitness Inc.,$18.00 - $21.00,30769231,Inc,19.5
98,GLGX,"Gerson Lehrman Group, Inc.",-,-,Inc,NaN


In [10]:
ipos_withdrawals['Shares Offered'].dtype

dtype('O')

In [11]:
ipos_withdrawals['Shares Offered'] = pd.to_numeric(ipos_withdrawals['Shares Offered'], errors='coerce')

In [12]:
ipos_withdrawals.isnull().sum()

Symbol             0
Company Name       0
Price Range        0
Shares Offered    28
Company Class      0
Avg. price        27
dtype: int64

In [13]:
ipos_withdrawals.dropna(inplace=True)

In [14]:
ipos_withdrawals.shape

(71, 6)

In [15]:
ipos_withdrawals['Withdrawn Value'] = ipos_withdrawals['Shares Offered'] * ipos_withdrawals['Avg. price']

In [16]:
ipos_withdrawals.shape

(71, 7)

In [17]:
withdrawn_by_class = ipos_withdrawals.groupby('Company Class')['Withdrawn Value'].sum()

In [18]:
withdrawn_by_class

Company Class
Acq.Corp    4.021000e+09
Group       3.378750e+07
Holdings    7.500000e+07
Inc         2.257164e+09
Limited     5.497346e+08
Other       7.679200e+08
Name: Withdrawn Value, dtype: float64

Question 1. What is the total withdrawn IPO value (in $ millions) for the company class with the highest total withdrawal value?
Answer: `Acq.Corp` with the total of 4021 millions

In [19]:
def get_ipos_by_year(year: int) -> pd.DataFrame:
    """
    Fetch IPO data for the given year from stockanalysis.com.
    """
    url = f"https://stockanalysis.com/ipos/{year}/"
    headers = {
        'User-Agent': (
            'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
            'AppleWebKit/537.36 (KHTML, like Gecko) '
            'Chrome/58.0.3029.110 Safari/537.3'
        )
    }

    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()

        # Wrap HTML text in StringIO to avoid deprecation warning
        # "Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object."
        html_io = StringIO(response.text)
        tables = pd.read_html(html_io)

        if not tables:
            raise ValueError(f"No tables found for year {year}.")

        return tables[0]

    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
    except ValueError as ve:
        print(f"Data error: {ve}")
    except Exception as ex:
        print(f"Unexpected error: {ex}")

    return pd.DataFrame()


In [20]:
ipos = get_ipos_by_year(2024)

In [21]:
ipos.shape

(225, 6)

In [22]:
ipos['IPO Date'] = pd.to_datetime(ipos['IPO Date'])

In [23]:
ipos['IPO Date'].unique()

<DatetimeArray>
['2024-12-31 00:00:00', '2024-12-27 00:00:00', '2024-12-23 00:00:00',
 '2024-12-20 00:00:00', '2024-12-19 00:00:00', '2024-12-18 00:00:00',
 '2024-12-13 00:00:00', '2024-12-12 00:00:00', '2024-12-10 00:00:00',
 '2024-12-06 00:00:00',
 ...
 '2024-02-01 00:00:00', '2024-01-31 00:00:00', '2024-01-26 00:00:00',
 '2024-01-25 00:00:00', '2024-01-24 00:00:00', '2024-01-19 00:00:00',
 '2024-01-18 00:00:00', '2024-01-12 00:00:00', '2024-01-11 00:00:00',
 '2024-01-09 00:00:00']
Length: 125, dtype: datetime64[ns]

In [24]:
ipos = ipos[ipos['IPO Date'] < '2024-06-01']

In [25]:
ipos['IPO Price'] = ipos['IPO Price'].str.replace('$', '', regex=False)

In [26]:
ipos.loc[:, 'IPO Price'] = pd.to_numeric(ipos['IPO Price'], errors='coerce')

In [27]:
ipos.isnull().sum()

IPO Date        0
Symbol          0
Company Name    0
IPO Price       2
Current         0
Return          0
dtype: int64

In [28]:
ipos.dropna(inplace=True)

In [29]:
ipos.shape

(75, 6)

In [30]:
ticker_list = ipos['Symbol'].tolist()

In [31]:
stocks_df = pd.DataFrame({'A' : []})

for i,ticker in enumerate(ticker_list):
  print(i,ticker)

  # Work with stock prices
  ticker_obj = yf.Ticker(ticker)

  # historyPrices = yf.download(tickers = ticker,
  #                    period = "max",
  #                    interval = "1d")
  historyPrices = ticker_obj.history(
                     period='max',
                     interval = "1d")

  # generate features for historical prices, and what we want to predict
  historyPrices['Ticker'] = ticker
  historyPrices['Year']= historyPrices.index.year
  historyPrices['Month'] = historyPrices.index.month
  historyPrices['Weekday'] = historyPrices.index.weekday
  historyPrices['Date'] = historyPrices.index.date

  # historical returns
  for i in [1,3,7,30,90,252,365]:
    historyPrices['growth_'+str(i)+'d'] = historyPrices['Close'] / historyPrices['Close'].shift(i)
  historyPrices['growth_future_30d'] = historyPrices['Close'].shift(-30) / historyPrices['Close']

  # Technical indicators
  # SimpleMovingAverage 10 days and 20 days
  historyPrices['SMA10']= historyPrices['Close'].rolling(10).mean()
  historyPrices['SMA20']= historyPrices['Close'].rolling(20).mean()
  historyPrices['growing_moving_average'] = np.where(historyPrices['SMA10'] > historyPrices['SMA20'], 1, 0)
  historyPrices['high_minus_low_relative'] = (historyPrices.High - historyPrices.Low) / historyPrices['Close']

  # 30d rolling volatility : https://ycharts.com/glossary/terms/rolling_vol_30
  historyPrices['volatility'] =   historyPrices['Close'].rolling(30).std() * np.sqrt(252)
  historyPrices['Sharpe'] = (historyPrices['growth_252d'] - 0.045) / historyPrices['volatility']
  # what we want to predict
  historyPrices['is_positive_growth_30d_future'] = np.where(historyPrices['growth_future_30d'] > 1, 1, 0)

  # sleep 1 sec between downloads - not to overload the API server
  time.sleep(1)


  if stocks_df.empty:
    stocks_df = historyPrices
  else:
    stocks_df = pd.concat([stocks_df, historyPrices], ignore_index=True)

0 BOW
1 HDL
2 RFAI
3 JDZG
4 RAY
5 BTOC
6 ZK
7 GPAT
8 PAL
9 SVCO
10 NNE
11 CCIX
12 VIK
13 ZONE
14 LOAR
15 MRX
16 RBRK
17 NCI
18 MFI
19 YYGH
20 TRSG
21 CDTG
22 CTRI
23 IBTA
24 MTEN
25 TWG
26 ULS
27 PACS
28 MNDR
29 CTNM
30 MAMO
31 ZBAO
32 BOLD
33 MMA
34 UBXG
35 IBAC
36 AUNA
37 BKHA
38 LOBO
39 RDDT
40 ALAB
41 INTJ
42 RYDE
43 LGCL
44 SMXT
45 VHAI
46 DYCQ
47 CHRO
48 UMAC
49 HLXB
50 MGX
51 TBBB
52 TELO
53 KYTX
54 PMNT
55 AHR
56 LEGT
57 ANRO
58 GUTS
59 AS
60 FBLG
61 AVBP
62 BTSG
63 HAO
64 CGON
65 YIBO
66 JL
67 SUGP
68 JVSA
69 KSPI
70 CCTG
71 PSBD
72 SYNX
73 SDHC
74 ROMA


In [32]:
stocks_df['Date'] = pd.to_datetime(stocks_df['Date'])
filtered_stocks = stocks_df[stocks_df['Date'].dt.date == pd.to_datetime('2025-06-06').date()]

In [33]:
filtered_stocks['growth_252d'].describe()

count    71.000000
mean      1.152897
std       1.406017
min       0.024970
25%       0.293422
50%       0.758065
75%       1.362736
max       8.097413
Name: growth_252d, dtype: float64

In [34]:
filtered_stocks['Sharpe'].describe()

count    71.000000
mean      0.288285
std       0.519028
min      -0.079677
25%       0.041215
50%       0.083768
75%       0.311507
max       2.835668
Name: Sharpe, dtype: float64

Question 2. What is the median Sharpe ratio (as of 6 June 2025) for companies that went public in the first 5 months of 2024? 
Answer:  0.083768

In [35]:
stocks_df

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Ticker,Year,Month,...,growth_252d,growth_365d,growth_future_30d,SMA10,SMA20,growing_moving_average,high_minus_low_relative,volatility,Sharpe,is_positive_growth_30d_future
0,23.000000,24.270000,22.139999,23.799999,3335800,0.0,0.0,BOW,2024,5,...,NaN,NaN,1.092017,NaN,NaN,0,0.089496,NaN,NaN,1
1,24.260000,26.150000,23.980000,25.700001,990500,0.0,0.0,BOW,2024,5,...,NaN,NaN,0.998054,NaN,NaN,0,0.084436,NaN,NaN,0
2,25.850000,26.879999,25.075001,26.480000,555100,0.0,0.0,BOW,2024,5,...,NaN,NaN,1.001133,NaN,NaN,0,0.068165,NaN,NaN,1
3,26.440001,26.490000,25.500999,26.290001,302700,0.0,0.0,BOW,2024,5,...,NaN,NaN,0.987828,NaN,NaN,0,0.037619,NaN,NaN,0
4,27.209999,27.209999,25.500000,26.139999,200900,0.0,0.0,BOW,2024,5,...,NaN,NaN,1.037490,NaN,NaN,0,0.065417,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23091,2.850000,3.310000,2.770000,2.770000,52500,0.0,0.0,ROMA,2025,6,...,5.368217,NaN,NaN,3.2600,2.70055,1,0.194946,11.810360,0.450724,0
23092,2.750000,3.153000,2.750000,3.000000,37400,0.0,0.0,ROMA,2025,6,...,6.000000,NaN,NaN,3.2610,2.74955,1,0.134333,11.401742,0.522289,0
23093,3.021000,3.050000,2.850000,2.880000,36300,0.0,0.0,ROMA,2025,6,...,5.938144,NaN,NaN,3.2390,2.78955,1,0.069444,10.886753,0.541313,0
23094,2.870000,2.890000,2.560000,2.660000,123100,0.0,0.0,ROMA,2025,6,...,4.666667,NaN,NaN,3.1970,2.82155,1,0.124060,10.383358,0.445103,0


In [36]:
periods = [{i:i*21} for i in range(1,13)] 

In [37]:
periods

[{1: 21},
 {2: 42},
 {3: 63},
 {4: 84},
 {5: 105},
 {6: 126},
 {7: 147},
 {8: 168},
 {9: 189},
 {10: 210},
 {11: 231},
 {12: 252}]

In [38]:
#df = pd.DataFrame()

In [39]:
for period_dict in periods:
    for months, days in period_dict.items():
        col_name = f'growth_future_{months}m'
        stocks_df[col_name] = stocks_df['Close'].shift(-days) / stocks_df['Close']


In [41]:
stocks_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23096 entries, 0 to 23095
Data columns (total 39 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   Open                           23096 non-null  float64       
 1   High                           23096 non-null  float64       
 2   Low                            23096 non-null  float64       
 3   Close                          23096 non-null  float64       
 4   Volume                         23096 non-null  int64         
 5   Dividends                      23096 non-null  float64       
 6   Stock Splits                   23096 non-null  float64       
 7   Ticker                         23096 non-null  object        
 8   Year                           23096 non-null  int32         
 9   Month                          23096 non-null  int32         
 10  Weekday                        23096 non-null  int32         
 11  Date           

In [43]:
min_date = stocks_df.groupby('Ticker')['Date'].min().reset_index()
min_date

,Ticker,Date
0,AHR,2024-02-07
1,ALAB,2024-03-20
2,ANRO,2024-02-02
3,AS,2024-02-01
4,AUNA,2024-03-22
...,...,...
70,YIBO,2024-01-25
71,YYGH,2024-04-22
72,ZBAO,2024-04-02
73,ZK,2024-05-10


In [44]:
merged_df = stocks_df.merge(min_date, on=['Ticker', 'Date'], how='inner')

In [45]:
merged_df.shape

(75, 39)

In [54]:
merged_df.iloc[:, -12:].dropna().describe()

,growth_future_1m,growth_future_2m,growth_future_3m,growth_future_4m,growth_future_5m,growth_future_6m,growth_future_7m,growth_future_8m,growth_future_9m,growth_future_10m,growth_future_11m,growth_future_12m
count,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000,75.000000
mean,0.927259,0.940749,0.833988,0.825192,0.803849,0.864267,0.847205,0.833046,0.881846,12.583912,12.583926,12.603037
std,0.346261,0.574267,0.409480,0.401772,0.488226,0.653078,0.712880,0.762355,0.936893,101.033895,101.336125,101.132531
min,0.098947,0.073800,0.060947,0.045368,0.054109,0.061432,0.048274,0.043103,0.033144,0.041357,0.023674,0.038947
25%,0.778984,0.685815,0.511212,0.517233,0.448403,0.384560,0.296870,0.208677,0.226740,0.245455,0.266821,0.229211
50%,0.977000,1.000000,0.927500,0.909091,0.821092,0.802239,0.844875,0.812109,0.822715,0.777871,0.719512,0.710647
75%,1.046509,1.154013,1.069085,1.134300,1.016381,1.093948,1.114468,1.082346,1.049719,1.244619,1.221407,1.320212
max,2.646505,4.874759,2.040000,1.605000,3.213873,3.670520,5.122350,5.171484,6.764933,875.862067,878.448251,876.724156


In [53]:
stocks_df.isnull().sum()

Open                                 0
High                                 0
Low                                  0
Close                                0
Volume                               0
Dividends                            0
Stock Splits                         0
Ticker                               0
Year                                 0
Month                                0
Weekday                              0
Date                                 0
growth_1d                           75
growth_3d                          225
growth_7d                          525
growth_30d                        2250
growth_90d                        6750
growth_252d                      18808
growth_365d                      23096
growth_future_30d                 2250
SMA10                              675
SMA20                             1425
growing_moving_average               0
high_minus_low_relative              0
volatility                        2175
Sharpe                   

In [56]:
# Rekordy z największym wzrostem
outliers = merged_df[merged_df['growth_future_12m'] > 5]
outliers[['Date', 'Ticker', 'Close', 'growth_future_12m']]


,Date,Ticker,Close,growth_future_12m
45,2024-09-11,VHAI,0.0116,876.724156


In [57]:
merged_df = merged_df[merged_df['Ticker'] != 'VHAI']

In [58]:
merged_df.iloc[:, -12:].dropna().describe()

,growth_future_1m,growth_future_2m,growth_future_3m,growth_future_4m,growth_future_5m,growth_future_6m,growth_future_7m,growth_future_8m,growth_future_9m,growth_future_10m,growth_future_11m,growth_future_12m
count,74.000000,74.000000,74.000000,74.000000,74.000000,74.000000,74.000000,74.000000,74.000000,74.000000,74.000000,74.000000
mean,0.933965,0.946763,0.839433,0.830635,0.810518,0.873768,0.857256,0.843720,0.893064,0.917991,0.883057,0.925724
std,0.343686,0.575804,0.409532,0.401721,0.488107,0.652297,0.712376,0.761894,0.938204,0.911383,0.862902,0.909959
min,0.098947,0.073800,0.060947,0.045368,0.054109,0.061432,0.048274,0.050746,0.033144,0.041357,0.023674,0.038947
25%,0.785894,0.696703,0.515089,0.521218,0.457946,0.408536,0.327393,0.234632,0.244048,0.242424,0.264661,0.228891
50%,0.982250,1.000000,0.932804,0.914441,0.823546,0.814453,0.848113,0.828492,0.833357,0.772592,0.717585,0.685001
75%,1.047153,1.161482,1.072380,1.138792,1.017101,1.098922,1.132689,1.086150,1.055767,1.200647,1.106726,1.186390
max,2.646505,4.874759,2.040000,1.605000,3.213873,3.670520,5.122350,5.171484,6.764933,5.352601,4.445545,4.849711


Question 3: What is the optimal number of months (1 to 12) to hold a newly IPO'd stock in order to maximize average growth?
Answer: 2